In [2]:
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
%matplotlib inline

In [3]:
# get data
pickle_file = 'data/notMNIST_clean_python3.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (197901, 28, 28) (197901,)
Validation set (9876, 28, 28) (9876,)
Test set (9857, 28, 28) (9857,)


### Reshape Data such that each example is a row

In [4]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

def reformat(dataset, labels):
    # as.type is not needed as the array is already float32 but just in case
    dataset = dataset.reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
    # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
    labels = (np.arange(num_labels) == labels[:, None]).astype(np.float32)
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)

print("Train Shapes --> Dataset: %s   Labels: %s" %(train_dataset.shape, train_labels.shape))
print("Valid Shapes --> Dataset: %s   Labels: %s" %(valid_dataset.shape, valid_labels.shape))
print("Test Shapes  --> Dataset: %s   Labels: %s" %(test_dataset.shape, test_labels.shape))

Train Shapes --> Dataset: (197901, 28, 28, 1)   Labels: (197901, 10)
Valid Shapes --> Dataset: (9876, 28, 28, 1)   Labels: (9876, 10)
Test Shapes  --> Dataset: (9857, 28, 28, 1)   Labels: (9857, 10)


In [5]:
def accuracy(predictions, labels):
    return 100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0]

### Network with two convolutional layers followed by one fully connected layer

In [16]:
batch_size = 16
patch_size = 5
depth1 = 16
depth2 = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():
    
    #Input data
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    patch1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth1], stddev=0.1))
    patch1_biases = tf.Variable(tf.zeros([depth1]))
    
    patch2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth1, depth2], stddev=0.1))
    patch2_biases = tf.Variable(tf.constant(1.0, shape=[depth2]))
    
    # divided by four because that is the size once the patches have scanned the image
    layer1_weights = tf.Variable(tf.truncated_normal(
                                 [image_size // 4 * image_size // 4 * depth2, num_hidden], stddev=0.1))
    layer1_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer2_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    # Model
    def model(data):
        # first convolution layer. Stride only matter in two elements in the middle
        conv1 = tf.nn.conv2d(data, patch1_weights, [1, 2, 2, 1], padding="SAME")
        conv1 = tf.nn.relu(conv1 + patch1_biases)
        
        # second convolution layer
        conv2 = tf.nn.conv2d(conv1, patch2_weights, [1, 2, 2, 1], padding="SAME")
        conv2 = tf.nn.relu(conv2 + patch2_biases)
        
        # reshape to apply fully connected layer
        shape_conv = conv2.get_shape().as_list()
        input_hidden = tf.reshape(conv2, [shape_conv[0], shape_conv[1] * shape_conv[2] * shape_conv[3]])
        hidden_layer = tf.nn.relu(tf.matmul(input_hidden, layer1_weights) + layer1_biases)
        return tf.matmul(hidden_layer, layer2_weights) + layer2_biases
    
    # training computation
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    sh = tf.shape(logits)
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))
    

In [17]:
%time
num_steps = 1001
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        # randomie offset
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 200 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 8.58 µs
Initialized
Minibatch loss at step 0: 2.706959
Minibatch accuracy: 6.2%
Validation accuracy: 10.1%
Minibatch loss at step 200: 1.140251
Minibatch accuracy: 87.5%
Validation accuracy: 76.4%
Minibatch loss at step 400: 0.721522
Minibatch accuracy: 75.0%
Validation accuracy: 78.3%
Minibatch loss at step 600: 0.525601
Minibatch accuracy: 87.5%
Validation accuracy: 80.0%
Minibatch loss at step 800: 0.349012
Minibatch accuracy: 87.5%
Validation accuracy: 81.7%
Minibatch loss at step 1000: 0.751198
Minibatch accuracy: 75.0%
Validation accuracy: 83.1%
Test accuracy: 89.5%


## Max Pool

In [18]:
batch_size = 16
patch_size = 5
depth1 = 16
depth2 = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():
    
    #Input data
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    patch1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth1], stddev=0.1))
    patch1_biases = tf.Variable(tf.zeros([depth1]))
    
    patch2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth1, depth2], stddev=0.1))
    patch2_biases = tf.Variable(tf.constant(1.0, shape=[depth2]))
    
    # divided by four because that is the size once the patches have scanned the image
    layer1_weights = tf.Variable(tf.truncated_normal(
                                 [image_size // 4 * image_size // 4 * depth2, num_hidden], stddev=0.1))
    layer1_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer2_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    # Model
    def model(data):
        # first convolution layer. Stride only matter in two elements in the middle
        conv1 = tf.nn.conv2d(data, patch1_weights, [1, 1, 1, 1], padding="SAME")
        conv1 = tf.nn.relu(conv1 + patch1_biases)
        conv1 = tf.nn.max_pool(conv1, [1, 2, 2, 1], [1, 2, 2, 1], padding="SAME" )
        
        
        # second convolution layer
        conv2 = tf.nn.conv2d(conv1, patch2_weights, [1, 1, 1, 1], padding="SAME")
        conv2 = tf.nn.relu(conv2 + patch2_biases)
        conv2 = tf.nn.max_pool(conv2, [1, 2, 2, 1], [1, 2, 2, 1], padding="SAME" )

        
        # reshape to apply fully connected layer
        shape_conv = conv2.get_shape().as_list()
        input_hidden = tf.reshape(conv2, [shape_conv[0], shape_conv[1] * shape_conv[2] * shape_conv[3]])
        hidden_layer = tf.nn.relu(tf.matmul(input_hidden, layer1_weights) + layer1_biases)
        return tf.matmul(hidden_layer, layer2_weights) + layer2_biases
    
    # training computation
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    sh = tf.shape(logits)
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))
    

In [19]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        # randomie offset
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 200 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.603429
Minibatch accuracy: 31.2%
Validation accuracy: 10.9%
Minibatch loss at step 100: 0.893740
Minibatch accuracy: 56.2%
Validation accuracy: 64.8%
Minibatch loss at step 200: 0.919437
Minibatch accuracy: 87.5%
Validation accuracy: 76.4%
Minibatch loss at step 300: 0.563234
Minibatch accuracy: 87.5%
Validation accuracy: 80.0%
Minibatch loss at step 400: 0.583443
Minibatch accuracy: 81.2%
Validation accuracy: 81.3%
Minibatch loss at step 500: 0.880446
Minibatch accuracy: 75.0%
Validation accuracy: 82.0%
Minibatch loss at step 600: 0.495690
Minibatch accuracy: 87.5%
Validation accuracy: 81.2%
Minibatch loss at step 700: 0.973824
Minibatch accuracy: 68.8%
Validation accuracy: 83.1%
Minibatch loss at step 800: 0.226850
Minibatch accuracy: 93.8%
Validation accuracy: 82.6%
Minibatch loss at step 900: 0.745880
Minibatch accuracy: 68.8%
Validation accuracy: 83.9%
Minibatch loss at step 1000: 0.532265
Minibatch accuracy: 75.0%
Validation accuracy: 84.4%

### Adding Dropout and Learning Decay Rate and Regularization and Increase Depth

In [11]:
batch_size = 128
patch_size = 5
depth1 = 16
depth2 = 32
num_hidden = 64

graph = tf.Graph()

with graph.as_default():
    
    #Input data
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    patch1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth1], stddev=0.1))
    patch1_biases = tf.Variable(tf.zeros([depth1]))
    
    patch2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth1, depth2], stddev=0.1))
    patch2_biases = tf.Variable(tf.constant(1.0, shape=[depth2]))
    
    # divided by four because that is the size once the patches have scanned the image
    layer1_weights = tf.Variable(tf.truncated_normal(
                                 [image_size // 4 * image_size // 4 * depth2, num_hidden], stddev=0.1))
    layer1_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer2_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    # Model
    def model(data, training):
        # first convolution layer. Stride only matter in two elements in the middle
        conv1 = tf.nn.conv2d(data, patch1_weights, [1, 1, 1, 1], padding="SAME")
        conv1 = tf.nn.relu(conv1 + patch1_biases)
        conv1 = tf.nn.max_pool(conv1, [1, 2, 2, 1], [1, 2, 2, 1], padding="SAME" )
        
        # second convolution layer
        conv2 = tf.nn.conv2d(conv1, patch2_weights, [1, 1, 1, 1], padding="SAME")
        conv2 = tf.nn.relu(conv2 + patch2_biases)
        conv2 = tf.nn.max_pool(conv2, [1, 2, 2, 1], [1, 2, 2, 1], padding="SAME" )

        # reshape to apply fully connected layer
        shape_conv = conv2.get_shape().as_list()
        input_hidden = tf.reshape(conv2, [shape_conv[0], shape_conv[1] * shape_conv[2] * shape_conv[3]])
        hidden_layer = tf.nn.relu(tf.matmul(input_hidden, layer1_weights) + layer1_biases)
        
        # adding dropout layer
        if training:
            hidden_layer = tf.nn.dropout(hidden_layer, 0.6)
        
        return tf.matmul(hidden_layer, layer2_weights) + layer2_biases
    
    # training computation
    logits = model(tf_train_dataset, True)
    regularization = tf.nn.l2_loss(layer1_weights) + tf.nn.l2_loss(layer2_weights)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + .0005 * regularization
    
    # Optimizer
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(0.05, global_step, 200, 0.95, staircase = True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset, False))
    test_prediction = tf.nn.softmax(model(tf_test_dataset, False))
    

In [12]:
num_steps = 10001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        # randomie offset
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 2000 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 6.578959
Minibatch accuracy: 11.7%
Validation accuracy: 10.1%
Minibatch loss at step 2000: 0.965237
Minibatch accuracy: 75.0%
Validation accuracy: 85.6%
Minibatch loss at step 4000: 0.883157
Minibatch accuracy: 82.8%
Validation accuracy: 86.3%
Minibatch loss at step 6000: 0.694455
Minibatch accuracy: 85.2%
Validation accuracy: 86.8%
Minibatch loss at step 8000: 0.695763
Minibatch accuracy: 85.2%
Validation accuracy: 86.9%
Minibatch loss at step 10000: 0.738749
Minibatch accuracy: 86.7%
Validation accuracy: 86.8%
Test accuracy: 93.0%


In [ ]:
batch_size = 128
patch_size = 5
depth1 = 16
depth2 = 32
num_hidden = 64

graph = tf.Graph()

with graph.as_default():
    
    #Input data
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    patch1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth1], stddev=0.1))
    patch1_biases = tf.Variable(tf.zeros([depth1]))
    
    patch2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth1, depth2], stddev=0.1))
    patch2_biases = tf.Variable(tf.constant(1.0, shape=[depth2]))
    
    # divided by four because that is the size once the patches have scanned the image
    layer1_weights = tf.Variable(tf.truncated_normal(
                                 [image_size // 4 * image_size // 4 * depth2, num_hidden], stddev=0.1))
    layer1_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer2_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    # Model
    def model(data, training):
        # first convolution layer. Stride only matter in two elements in the middle
        conv1 = tf.nn.conv2d(data, patch1_weights, [1, 1, 1, 1], padding="SAME")
        conv1 = tf.nn.relu(conv1 + patch1_biases)
        conv1 = tf.nn.max_pool(conv1, [1, 2, 2, 1], [1, 2, 2, 1], padding="SAME" )
        
        # second convolution layer
        conv2 = tf.nn.conv2d(conv1, patch2_weights, [1, 1, 1, 1], padding="SAME")
        conv2 = tf.nn.relu(conv2 + patch2_biases)
        conv2 = tf.nn.max_pool(conv2, [1, 2, 2, 1], [1, 2, 2, 1], padding="SAME" )

        # reshape to apply fully connected layer
        shape_conv = conv2.get_shape().as_list()
        input_hidden = tf.reshape(conv2, [shape_conv[0], shape_conv[1] * shape_conv[2] * shape_conv[3]])
        hidden_layer = tf.nn.relu(tf.matmul(input_hidden, layer1_weights) + layer1_biases)
        
        # adding dropout layer
        if training:
            hidden_layer = tf.nn.dropout(hidden_layer, 0.6)
        
        return tf.matmul(hidden_layer, layer2_weights) + layer2_biases
    
    # training computation
    logits = model(tf_train_dataset, True)
    regularization = tf.nn.l2_loss(layer1_weights) + tf.nn.l2_loss(layer2_weights)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + .0005 * regularization
    
    # Optimizer
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(0.05, global_step, 2000, 0.95, staircase = True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset, False))
    test_prediction = tf.nn.softmax(model(tf_test_dataset, False))
    

In [ ]:
# decay_step changed to 100 instead of 1000

num_steps = 100001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        # randomie offset
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 20000 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))